### 소비자 행동데이터 분석을 위한 2019년 10월 1개월 간의 소비자 행동데이터

In [ ]:
# #Elasticsearch 설치
# !pip install elasticsearch

In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

In [2]:
# 전체 데이터 읽기
file_path = r"/home/user1/project/shop_Modeling/data/shopping_data/2019-Oct.csv"
chunk_size = 5000
chunks = [] # 데이터를 청크 단위로 읽어오기
num_rows_to_read = 3000000  # 읽을 행의 총 수
rows_read = 0

In [3]:
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    # 여기서 각 청크에 대한 원하는 처리 수행
    # 예를 들어, 데이터프레임에 저장하거나 다른 작업을 수행

    # 현재 예제에서는 각 청크를 리스트에 추가
    chunks.append(chunk)
    rows_read += chunk.shape[0]

    # 읽은 행의 총 수가 목표 행의 수에 도달하면 중단
    if rows_read >= num_rows_to_read:
        break

# 모든 청크를 하나의 데이터프레임으로 통합
data = pd.concat(chunks, ignore_index=True)

In [ ]:
# # Pandas의 read_csv 함수를 이용해 청크 단위로 데이터 읽어오기
# for chunk in pd.read_csv(file_path, chunksize=chunk_size, iterator=True):
#     data_frames.append(chunk)
    
#     # 읽어온 데이터의 행 수가 목표 행 수에 도달하면 종료
#     if len(chunk) >= total_rows:
#         break

# # 모든 청크를 병합하여 최종 DataFrame 생성
# data = pd.concat(data_frames, ignore_index=True)

In [4]:
# 데이터프레임의 크기 확인
print("데이터프레임의 크기:", data.shape)

데이터프레임의 크기: (3000000, 9)


In [5]:
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [ ]:
# # 큰 크기의 CSV 파일을 더 작은 청크(chunk)로 나누어 읽고 이를 데이터프레임에 추가
# chunks = pd.read_csv(file_path, encoding='utf-8', chunksize=1000)

In [6]:
# # Elasticsearch 서버 연결
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

In [7]:
# Elasticsearch 서버에 ping 요청 보내기
if es.ping():
    print("Elasticsearch 서버에 성공적으로 연결되었습니다.")
else:
    print("Elasticsearch 서버에 연결하지 못했습니다.")

Elasticsearch 서버에 성공적으로 연결되었습니다.


In [8]:
# Elasticsearch에 매핑 설정
shopping_oct = 'shopping_oct'  # 원하는 색인명으로 변경

# 인덱스 매핑 설정
mapping = {
    "mappings": {
        "properties": {
            "event_time": {"type": "date", "format": "yyyy-MM-dd HH:mm:ss Z"},  # 날짜 필드
            "event_type": {"type": "keyword"},  # 문자열 키워드 필드
            "product_id": {"type": "keyword"},  # 문자열 키워드 필드
            "category_id": {"type": "keyword"},  # 문자열 키워드 필드
            "category_code": {"type": "keyword"},  # 문자열 키워드 필드
            "brand": {"type": "keyword"},  # 문자열 키워드 필드
            "price": {"type": "float"},  # 부동소수점 숫자 필드
            "user_id": {"type": "keyword"},  # 문자열 키워드 필드
            "user_session": {"type": "keyword"}  # 문자열 키워드 필드
        }
    }
}

In [9]:
# 매핑을 설정한 인덱스에 적용
es.indices.create(index=shopping_oct, body=mapping, ignore=400)

/tmp/ipykernel_13326/3024227504.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=shopping_oct, body=mapping, ignore=400)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'shopping_oct'})

In [ ]:
# # 큰 크기의 CSV 파일을 더 작은 청크(chunk)로 나누어 읽고 이를 데이터프레임에 추가
# chunks = pd.read_csv(file_path, encoding='utf-8', chunksize=1000)

In [10]:
# Elasticsearch에 이미 색인된 데이터 삭제
es.indices.delete(index=shopping_oct, ignore=[400, 404])

/tmp/ipykernel_13326/553074156.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index=shopping_oct, ignore=[400, 404])


ObjectApiResponse({'acknowledged': True})

In [ ]:
# 큰 크기의 CSV 파일을 작은 청크(chunk)로 나누어 읽고 이를 Elasticsearch에 색인
chunk_size = 1000
for chunk in pd.read_csv(file_path, encoding='utf-8', chunksize=chunk_size):
    # 각 청크에 대한 처리를 여기에 추가
    # 예를 들어, 원하는 계산 수행 또는 데이터 전처리 등을 수행

    # NaN을 0으로 대체
    chunk = chunk.fillna(0)

    # Elasticsearch에 색인할 때 사용할 데이터프레임 생성
    for i, row in chunk.iterrows():
        document = row.to_dict()

        # 'brand'와 'category_code' 필드를 명시적으로 문자열로 변환
        document['brand'] = str(document['brand'])
        document['category_code'] = str(document['category_code'])

        try:
            # Elasticsearch에 색인
            es.index(index=shopping_oct, body=document)
        except Exception as e:
            print(f"Error indexing document: {e}")
            print(f"Error details: {document}")

print("색인이 완료되었습니다.")

In [ ]:
import elasticsearch
print(elasticsearch.__version__)